In [1]:
import numpy as np

import tensorflow.keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Reshape
from tensorflow.keras.layers import LSTM, Bidirectional, Embedding
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split

import pickle as pk

BATCH_SIZE = 32
EPOCHS = 70
VALIDATION_SPLIT = 0.02

train_data_path = "train_data.pk"

2023-12-15 09:34:27.652675: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
train_data = pk.load(open(train_data_path, "rb"))
X_train = train_data["X"]
X_train_oh = train_data["X_oh"]

alphabet_size = train_data["alphabet_size"]

len(X_train), len(X_train_oh)

(4855, 4855)

In [3]:
X_train, X_val, X_train_oh, X_val_oh = train_test_split(X_train, X_train_oh, test_size=VALIDATION_SPLIT)

In [4]:
def get_model(input_shape, alphabet_size):
    OUTPUT_DIM = alphabet_size # sigmoid output

    input_layer = Input(shape=input_shape)
    x = Embedding(alphabet_size, 4)(input_layer)
    x = Bidirectional(LSTM(8))(x)
    x = Dense(OUTPUT_DIM, activation="softmax")(x)
    model = Model(input_layer, x)

    model.compile(
        loss="categorical_crossentropy",
        optimizer="adam",
        metrics=["categorical_crossentropy"]
    )
    return model

In [5]:
model = get_model((1,), alphabet_size)

model(np.array(X_train[0])).shape, alphabet_size

(TensorShape([20, 30]), 30)

In [6]:
# because we have arrays of different length we have to use the fit_generator() method

def data_generator():
    global X_train
    global X_train_oh
    
    idx = 0
    while True:
        if idx == len(X_train):
            idx = 0
        yield np.array(X_train[idx]), np.array(X_train_oh[idx])
        idx += 1

def val_generator():
    global X_val
    global X_val_oh
    
    idx = 0
    while True:
        if idx == len(X_val):
            idx = 0
        yield np.array(X_val[idx]), np.array(X_val_oh[idx])
        idx += 1

In [ ]:
es = EarlyStopping(
    monitor="val_loss",
    patience=5,
    mode="auto")

data_gen = data_generator()
val_gen = val_generator()

model.fit(
    data_gen,
    steps_per_epoch=int(len(X_train) / BATCH_SIZE),
    epochs=EPOCHS,
    validation_data=val_gen,
    validation_steps=len(X_val),
    callbacks=[es]
)

Epoch 1/70
148/148 [==============================] - 4s 8ms/step - loss: 3.1890 - categorical_crossentropy: 3.1890 - val_loss: 2.8011 - val_categorical_crossentropy: 2.8011
Epoch 2/70
148/148 [==============================] - 0s 3ms/step - loss: 2.3575 - categorical_crossentropy: 2.3575 - val_loss: 2.1164 - val_categorical_crossentropy: 2.1164
Epoch 3/70
148/148 [==============================] - 0s 3ms/step - loss: 1.9452 - categorical_crossentropy: 1.9452 - val_loss: 1.8015 - val_categorical_crossentropy: 1.8015
Epoch 4/70
148/148 [==============================] - 0s 3ms/step - loss: 1.7020 - categorical_crossentropy: 1.7020 - val_loss: 1.6331 - val_categorical_crossentropy: 1.6331
Epoch 5/70
148/148 [==============================] - 0s 3ms/step - loss: 1.5313 - categorical_crossentropy: 1.5313 - val_loss: 1.3489 - val_categorical_crossentropy: 1.3489
Epoch 6/70
148/148 [==============================] - 0s 3ms/step - loss: 1.1481 - categorical_crossentropy: 1.1481 - val_loss: 1.

In [ ]:
model.save("model.keras")